ETL PROJECT BY MELINDA EUDY, KELLY ROSE, DARRIUS SUMMERS MARCH 2021

In [47]:
#dependencies
import pandas as pd
import requests
import json
from pprint import pprint
from config import api_key
from sqlalchemy import create_engine

api_key= api_key
headers = {'Authorization': 'Bearer %s' % api_key}
url='https://api.yelp.com/v3/businesses/search'

STORE CSV INTO DATAFRAME ---KAGGLE - BREWERIES AND BEER PUBS IN THE US

In [48]:
csv_file = "Resources/7160_1.csv"
breweries_data_df = pd.read_csv(csv_file)
breweries_data_df.head()

,address,categories,city,country,key,lat,long,name,phones,postalCode,province,websites
0,407 Radam Ln,brewery,Austin,US,us/tx/austin/407radamln,NaN,NaN,(512) Brewing Co,5127072337,78745,TX,NaN
1,1135 N W Galveston Ave,Brewery and Bar,Bend,US,us/or/bend/1135nwgalvestonave,NaN,NaN,"10 Barrel Brewing Company, 10 Barrel Brewing Co","(541) 585-1007, 5415851007",97703,OR,10barrel.com
2,830 W Bannock St,"brewery, Restaurant, Pub, Brewery, pub, Brewer...",Boise,US,us/id/boise/830wbannockst,43.617711,-116.202883,"10 Barrel Brewing, 10 Barrel Brewing Co.","(208) 344-5870, 2083445870, 2.08344587E9",83702,ID,10barrel.com
3,1022 Texan Trl,"Wine Tours & Tastings, Food & Drink, Tours, Br...",Grapevine,US,us/tx/grapevine/1022texantrl,32.938385,-97.064340,10 Gallon Tours,"18174031832, 817 403-1832",76051,TX,http://www.10gallontours.com/
4,Po Box 135,"American Restaurants, Bar & Grill Restaurants,...",Villard,US,us/mn/villard/pobox135,NaN,NaN,10 Mile Tavern,3205542939,56385,MN,http://www.10miletavern.com


In [50]:
# Remove locations with missing data
breweries_data_df.dropna()
breweries_data_df.head()

,address,categories,city,country,key,lat,long,name,phones,postalCode,province,websites
0,407 Radam Ln,brewery,Austin,US,us/tx/austin/407radamln,NaN,NaN,(512) Brewing Co,5127072337,78745,TX,NaN
1,1135 N W Galveston Ave,Brewery and Bar,Bend,US,us/or/bend/1135nwgalvestonave,NaN,NaN,"10 Barrel Brewing Company, 10 Barrel Brewing Co","(541) 585-1007, 5415851007",97703,OR,10barrel.com
2,830 W Bannock St,"brewery, Restaurant, Pub, Brewery, pub, Brewer...",Boise,US,us/id/boise/830wbannockst,43.617711,-116.202883,"10 Barrel Brewing, 10 Barrel Brewing Co.","(208) 344-5870, 2083445870, 2.08344587E9",83702,ID,10barrel.com
3,1022 Texan Trl,"Wine Tours & Tastings, Food & Drink, Tours, Br...",Grapevine,US,us/tx/grapevine/1022texantrl,32.938385,-97.064340,10 Gallon Tours,"18174031832, 817 403-1832",76051,TX,http://www.10gallontours.com/
4,Po Box 135,"American Restaurants, Bar & Grill Restaurants,...",Villard,US,us/mn/villard/pobox135,NaN,NaN,10 Mile Tavern,3205542939,56385,MN,http://www.10miletavern.com


In [51]:
# Delete Columns not required for final production database
del breweries_data_df["key"]
del breweries_data_df["long"]
del breweries_data_df["lat"]
del breweries_data_df["categories"]

# Reorder the columns to create better logic
updated_breweries_data_df = breweries_data_df[["name", "phones", "address", "city", "province", "postalCode", "websites"]]
updated_breweries_data_df.head()

,name,phones,address,city,province,postalCode,websites
0,(512) Brewing Co,5127072337,407 Radam Ln,Austin,TX,78745,NaN
1,"10 Barrel Brewing Company, 10 Barrel Brewing Co","(541) 585-1007, 5415851007",1135 N W Galveston Ave,Bend,OR,97703,10barrel.com
2,"10 Barrel Brewing, 10 Barrel Brewing Co.","(208) 344-5870, 2083445870, 2.08344587E9",830 W Bannock St,Boise,ID,83702,10barrel.com
3,10 Gallon Tours,"18174031832, 817 403-1832",1022 Texan Trl,Grapevine,TX,76051,http://www.10gallontours.com/
4,10 Mile Tavern,3205542939,Po Box 135,Villard,MN,56385,http://www.10miletavern.com


In [52]:
# Fix column names to prepare for merge with Yelp data
updated_breweries_data_df=updated_breweries_data_df.rename(columns ={"province": "state", "postalCode": "zip_code", "address": "address1"})
updated_breweries_data_df.head()

,name,phones,address1,city,state,zip_code,websites
0,(512) Brewing Co,5127072337,407 Radam Ln,Austin,TX,78745,NaN
1,"10 Barrel Brewing Company, 10 Barrel Brewing Co","(541) 585-1007, 5415851007",1135 N W Galveston Ave,Bend,OR,97703,10barrel.com
2,"10 Barrel Brewing, 10 Barrel Brewing Co.","(208) 344-5870, 2083445870, 2.08344587E9",830 W Bannock St,Boise,ID,83702,10barrel.com
3,10 Gallon Tours,"18174031832, 817 403-1832",1022 Texan Trl,Grapevine,TX,76051,http://www.10gallontours.com/
4,10 Mile Tavern,3205542939,Po Box 135,Villard,MN,56385,http://www.10miletavern.com


In [53]:
# Create dictornary search from YELP for bars in North Carolina
params = {'term':'bars','location':'NC'}

# Making a get request to the API
response=requests.get(url, params=params, headers=headers)

# proceed only if the status code is 200
print('The status code is {}'.format(response.status_code))


The status code is 200


In [54]:
# Print response from YELP API for Bars in North Carolina
data = response.json()
pprint(data)

{'businesses': [{'alias': 'the-fairwood-226-charlotte-2',
                 'categories': [{'alias': 'beerbar', 'title': 'Beer Bar'},
                                {'alias': 'sportsbars', 'title': 'Sports Bars'},
                                {'alias': 'chicken_wings',
                                 'title': 'Chicken Wings'}],
                 'coordinates': {'latitude': 35.20268, 'longitude': -80.87114},
                 'display_phone': '(980) 309-0463',
                 'distance': 3749.4149093062024,
                 'id': 'VlTR8dhK2Cz6N5kHB6DKRw',
                 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/l7AtTYVpUJrmOfE2LdhMEQ/o.jpg',
                 'is_closed': False,
                 'location': {'address1': '226 Fairwood Ave',
                              'address2': None,
                              'address3': '',
                              'city': 'Charlotte',
                              'country': 'US',
                              'display_addr

In [62]:
# create dataframe for YELP bars in North Carolina
bars_nc_dict = {
    "name": (data[businessess][0][name]), 
    "Yelp price": [businessess][0][price],
    "Yelp rating": [businessess][0][rating],
    "Yelp review count": [businessess][0][review_count]
}
bars_nc = pd.DataFrame(bars_nc_dict)
bars_nc.head()

NameError: name 'businessess' is not defined

In [14]:
# Create dictornary search from YELP for bars in South Carolina
params = {'term':'bars','location':'SC'}

# Making a get request to the API
response2=requests.get(url, params=params, headers=headers)

# proceed only if the status code is 200
print('The status code is {}'.format(response2.status_code))

The status code is 200


In [15]:
# Print response from YELP API
data = response2.json()
pprint(data)

{'businesses': [{'alias': 'the-whig-columbia',
                 'categories': [{'alias': 'divebars', 'title': 'Dive Bars'}],
                 'coordinates': {'latitude': 34.0015114,
                                 'longitude': -81.033505},
                 'display_phone': '(803) 931-8852',
                 'distance': 4144.845632578871,
                 'id': 'eL-gvkAzpoGHMmgKURSJDw',
                 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/o2ir-8jXxMlbG8QhoB3OSw/o.jpg',
                 'is_closed': False,
                 'location': {'address1': '1200 Main St',
                              'address2': '',
                              'address3': '',
                              'city': 'Columbia',
                              'country': 'US',
                              'display_address': ['1200 Main St',
                                                  'Columbia, SC 29201'],
                              'state': 'SC',
                              'zip_code'